In [1]:
#imports Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup

#imports ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager

#imports pandas
import pandas as pd

In [2]:
#sets the executable path and initializes the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

browser = Browser('chrome', **executable_path, headless = False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




### TITLE AND BODY

In [3]:
#visits the mars nasa website
url = 'http://mars.nasa.gov/news/'
browser.visit(url)

#optional delay for loading the page
#searching for elements with specific combinations, ul tag w/ item_list attributes and li tag with slide attribute
browser.is_element_present_by_css('ul.item_list li.slide', wait_time = 1)

True

In [4]:
#gets the html content of the page
html = browser.html

#parses html page
news_soup = soup(html, 'html.parser')

#creates the first search
#grabs the first item that matches both arguments
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
#extracts only the text title
news_title = slide_elem.find('div', class_= 'content_title').get_text()
news_title

"NASA's Perseverance Drives on Mars' Terrain for First Time"

In [6]:
#extracts only the article body summary
news_p = slide_elem.find('div', class_= 'article_teaser_body').get_text()
news_p

'The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.'

### FEATURED IMAGES

In [7]:
#visits the website
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [8]:
#find and click full image button
full_image_elem= browser.find_by_tag('button')[1]
full_image_elem.click()

In [9]:
#parses the resulting html with soup
html = browser.html

img_soup = soup(html, 'html.parser')

In [10]:
#extracting image
image_url_rel = img_soup.find('img', class_ = 'fancybox-image').get('src')
image_url_rel #only partial url

#gets base url to create absolute url 
img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{image_url_rel}'
img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

### FACTS

In [11]:
#extracting the html table and convert it into a pandas dataframe
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns = ['description', 'value']
df.set_index('description', inplace = True)

df.to_html()
#likewise the dataframe can also be converted back into html for uploading into a new web page

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

###

### HIGH RESOLUTION MARS HEMISPHERE IMAGES AND TITLES

In [12]:
#visits the browser 1.
main_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
main_enhanced = 'https://astrogeology.usgs.gov'

browser.visit(main_url)

In [13]:
#creates a list to hold the image url (string) and titles 2.
hemisphere_image_urls = []
hemisphere_titles = []

html = browser.html
hemisphere_soup = soup(html, 'html.parser')
main_elem = hemisphere_soup.find_all('div', class_='item')


#retrieves the image urls and titles for each hemisphere 3.
for title_img in main_elem:
    #gets the title from first page
    hemisphere_titles =title_img.find('h3').get_text()

    #builds the url for the second webpage to find full image url
    part_enhanced = title_img.find('a')['href']
    enhanced = main_enhanced + part_enhanced

    #parses the enhanced webpage to get the URL
    browser.visit(enhanced)
    html2 = browser.html
    enhanced_soup = soup(html2, 'html.parser')

    downloads_elem = enhanced_soup.find('div', class_ = 'downloads')
    image_urls = downloads_elem.find('a')['href']

    #dictionary to hold title and images
    hemisphere = dict({'Images': image_urls, 'Titles': hemisphere_titles})
    hemisphere_image_urls.append(hemisphere)

In [14]:
#prints the list holding image url(string) and titles 4.
hemisphere_image_urls


[{'Images': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'Titles': 'Cerberus Hemisphere Enhanced'},
 {'Images': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'Titles': 'Schiaparelli Hemisphere Enhanced'},
 {'Images': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'Titles': 'Syrtis Major Hemisphere Enhanced'},
 {'Images': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'Titles': 'Valles Marineris Hemisphere Enhanced'}]

In [15]:
#quits session 5.
browser.quit()